In [1]:
import numpy as np
import pandas as pd
import re

## READ THE DATA

In [2]:
fileName='input.txt'
dockingList = []

with open(fileName) as openFileObject:
    for line in openFileObject:
        header, argument = line.rstrip().split(sep=' = ')
        if header == 'mask':
            mask = argument
        else:
            address = re.split('\[|\]', header)[1]
            dockingList.append([mask, int(address), int(argument)])

dockingData = pd.DataFrame(dockingList, columns=['mask','address', 'number'])
dockingData.head()

,mask,address,number
0,1X11X010X000X0X101X00100011X10100111,40278,36774405
1,1X11X010X000X0X101X00100011X10100111,51306,55175378
2,1X11X010X000X0X101X00100011X10100111,31036,805355
3,1X11X010X000X0X101X00100011X10100111,8433,48148
4,1X11X010X000X0X101X00100011X10100111,58481,45466


## PART 1

In [3]:
def ConvertToBinary(number):
    binary = str("{0:b}".format(number))
    if len(binary) > 36:
        print("ERROR: number", number,"requires more than 36 bits to store as binary.")
    
    binary = binary.zfill(36)
    return binary

def ApplyMask(number, mask):
    newNumber = ''
    
    for i in range(36):
        if mask[i] != 'X':
            newNumber = newNumber + str(mask[i])
        else:
            newNumber = newNumber + str(number[i])
    
    return newNumber

In [4]:
memory = {}

for i in dockingData.index.values:
    mask = dockingData.loc[i, 'mask']
    address = dockingData.loc[i, 'address']
    number = dockingData.loc[i, 'number']
    
    memory[address] = int(ApplyMask(ConvertToBinary(number),mask),2)

print("The sum of all values stored in memory is", sum(memory.values()),".")

The sum of all values stored in memory is 16003257187056 .


## PART 2

In [5]:
def ApplyMaskGVersion2(number, mask):
    newNumber = ''
    
    for i in range(36):
        if mask[i] == '0':
            newNumber = newNumber + str(number[i])
        elif mask[i] == '1':
            newNumber = newNumber + '1'
        elif mask[i] == 'X':
            newNumber = newNumber + 'X'
        else:
            print("Why is there a", mask[i], "in my mask?")
            
    
    return newNumber

def FillFloatingBits(binaryString):
    ## This base also takes care of the case in which binaryString is empty
    indexOfFirstX = binaryString.find('X')
    if indexOfFirstX == -1:
        return [binaryString]
    
    leftOfX = binaryString[0:indexOfFirstX]
    rightOfX = binaryString[indexOfFirstX+1:len(binaryString)]
    
    filledStrings = []
    
    for filledRightOfX in FillFloatingBits(rightOfX):
        filledStrings.append(leftOfX + '0' + filledRightOfX)
        filledStrings.append(leftOfX + '1' + filledRightOfX)
        
    return filledStrings

In [6]:
memoryVersion2 = {}

for i in dockingData.index.values:
    mask = dockingData.loc[i, 'mask']
    address = dockingData.loc[i, 'address']
    number = dockingData.loc[i, 'number']
    
    maskedAddress = ApplyMaskGVersion2(ConvertToBinary(address), mask)
    newAddressBinaryList = FillFloatingBits(maskedAddress)
    
    for newAddressBinary in newAddressBinaryList:
        memoryVersion2[int(newAddressBinary,2)] = number

print("The (new) sum of all values stored in memory is", sum(memoryVersion2.values()),".")

The (new) sum of all values stored in memory is 3219837697833 .
